In [1]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning

In [2]:
alpha_df = pd.read_csv("./shannon_diversity_metadata.tsv", sep = "\t", index_col = "id")
alpha_df = alpha_df.drop("#q2:types")

alpha_df["tissue_types"] = alpha_df["tissue_type"].astype("category")
alpha_df["study_title"] = alpha_df["study_title"].astype("category")
alpha_df["fixation_method"] = alpha_df["fixation_method"].astype("category")
alpha_df["shannon_entropy"] = alpha_df["shannon_entropy"].astype(str).astype(float)

print(alpha_df.dtypes)

biosamplemodel                     object
collection_date                    object
control_vs_patient                 object
ebi_metadata_retrieved             object
ena_first_public                   object
ena_last_update                    object
env_material                       object
experiment_design_description      object
experiment_title                   object
experiment_title_specific          object
geo_loc_name                       object
host                               object
instrument_model                   object
lat_lon                            object
library_layout                     object
library_name                       object
library_source                     object
primary_experimental_variable      object
qebil_prep_file                    object
qiita_study_alias                  object
qiita_study_title                  object
sample_accession                   object
sample_alias                       object
sample_site                       

In [3]:
def read_in_data(input_file, target_feature):
    df = pd.read_csv(input_file, sep = "\t", index_col = "id")
    df = df.drop("#q2:types")

    df["tissue_types"] = df["tissue_type"].astype("category")
    df["study_title"] = df["study_title"].astype("category")
    df["fixation_method"] = df["fixation_method"].astype("category")
    df[target_feature] = df[target_feature].astype(str).astype(float)
    return df
    
def create_model(input_file, target_feature):

    feature_table = read_in_data(input_file, target_feature)
    metadata_technical = 'study_title + fixation_method'
    metadata_biological = 'tissue_type'
    
    anova_results = {}

    lm = ols(target_feature + '~'  + metadata_biological + '+' + metadata_technical, data = feature_table).fit()
    print(lm.summary())
    table = sm.stats.anova_lm(lm, typ = 2)
    print(table)
    anova_results.update({target_feature:table.sort_values(by = 'PR(>F)')})

    print(anova_results)

In [4]:
cat_columns = alpha_df.select_dtypes(['category']).columns
print(cat_columns)

alpha_df[cat_columns] = alpha_df[cat_columns].apply(lambda x: x.cat.codes)
alpha_df["tissue_type"]

Index(['study_title', 'fixation_method', 'tissue_types'], dtype='object')


id
178664.14099.SAMN16262001           pancreatic
178702.14146.SAMN09908085    colorectal mucosa
178702.14146.SAMN09908089    colorectal mucosa
178702.14146.SAMN09908071    colorectal mucosa
178718.14099.SAMN16262005           pancreatic
                                   ...        
179088.14191.SAMN16691961       gastric mucosa
179088.14191.SAMN16691960       gastric mucosa
178770.14140.SAMN12261653    easophagal mucosa
178770.14140.SAMN12261648    easophagal mucosa
178770.14140.SAMN12261651    easophagal mucosa
Name: tissue_type, Length: 292, dtype: object

In [5]:
alpha_metrics_anova = 'shannon_entropy'
metadata_technical = 'study_title + fixation_method'
metadata_biological = 'tissue_type'

In [6]:
anova_results = {}

lm = ols(alpha_metrics_anova + '~' + metadata_biological + '+' + metadata_technical, data = alpha_df).fit()
table = sm.stats.anova_lm(lm, typ = 2)
anova_results.update({alpha_metrics_anova:table.sort_values(by = 'PR(>F)')})

print(anova_results)

{'shannon_entropy':                      sum_sq     df          F        PR(>F)
tissue_type      716.984931    8.0  50.188521  7.072539e-50
study_title        6.608485    1.0   3.700720  5.539905e-02
fixation_method    0.629843    1.0   0.352709  5.530614e-01
Residual         501.789953  281.0        NaN           NaN}


In [7]:
create_model("shannon_diversity_metadata.tsv", "shannon_entropy")

                            OLS Regression Results                            
Dep. Variable:        shannon_entropy   R-squared:                       0.832
Model:                            OLS   Adj. R-squared:                  0.822
Method:                 Least Squares   F-statistic:                     84.94
Date:                Fri, 15 Dec 2023   Prob (F-statistic):           2.40e-96
Time:                        10:20:45   Log-Likelihood:                -419.09
No. Observations:                 292   AIC:                             872.2
Df Residuals:                     275   BIC:                             934.7
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                                                                                                                       coef    std err          t      P>|t|      [0.025      0.975]
-------------

In [8]:
create_model("evenness_metadata.tsv","pielou_evenness")

                            OLS Regression Results                            
Dep. Variable:        pielou_evenness   R-squared:                       0.871
Model:                            OLS   Adj. R-squared:                  0.864
Method:                 Least Squares   F-statistic:                     116.4
Date:                Fri, 15 Dec 2023   Prob (F-statistic):          2.99e-112
Time:                        10:20:45   Log-Likelihood:                 271.09
No. Observations:                 292   AIC:                            -508.2
Df Residuals:                     275   BIC:                            -445.7
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                                                                                                                       coef    std err          t      P>|t|      [0.025      0.975]
-------------

In [9]:
create_model("faithpd_metadata.tsv","faith_pd")

                            OLS Regression Results                            
Dep. Variable:               faith_pd   R-squared:                       0.786
Model:                            OLS   Adj. R-squared:                  0.773
Method:                 Least Squares   F-statistic:                     63.11
Date:                Fri, 15 Dec 2023   Prob (F-statistic):           3.73e-82
Time:                        10:20:45   Log-Likelihood:                -1247.4
No. Observations:                 292   AIC:                             2529.
Df Residuals:                     275   BIC:                             2591.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                                                                                                                       coef    std err          t      P>|t|      [0.025      0.975]
-------------